# Partition Overview

In [38]:
 DECLARE @SchemaName nvarchar(100) = 'Sales', @TableName nvarchar(100) = 'Orders_Part'
 DECLARE @ShowPartNr int = 0

 SELECT
    --sc.name + N'.' + so.name as [Schema.Table],
    --si.index_id as [Index ID],
    --si.type_desc as [Structure],
    si.name as [Index],
    stat.row_count AS [Rows],
    CONVERT(decimal(8,3), stat.in_row_reserved_page_count * 8./1024./1024.) as [In-Row GB],
    --stat.lob_reserved_page_count * 8./1024./1024. as [LOB GB],
    p.partition_number AS [Partition #],
	p.data_compression_desc AS [Compression],
    pf.name as [Partition Function],
	lv.value leftValue,
    rv.value rightValue,
    CASE pf.boundary_value_on_right	
        WHEN 1 then 'Right / Lower'
        ELSE 'Left / Upper'
    END as [Boundary Type],
    prv.value as [Boundary Point (date)],
    fg.name as [Filegroup]
FROM sys.partition_functions AS pf
JOIN sys.partition_schemes as ps on ps.function_id=pf.function_id
JOIN sys.indexes as si on si.data_space_id=ps.data_space_id
JOIN sys.objects as so on si.object_id = so.object_id
JOIN sys.schemas as sc on so.schema_id = sc.schema_id
JOIN sys.partitions as p on 
    si.object_id=p.object_id 
    and si.index_id=p.index_id
LEFT JOIN sys.partition_range_values as prv on prv.function_id=pf.function_id
    and p.partition_number= 
        CASE pf.boundary_value_on_right WHEN 1
            THEN prv.boundary_id + 1
        ELSE prv.boundary_id
        END
        /* For left-based functions, partition_number = boundary_id, 
           for right-based functions we need to add 1 */
LEFT JOIN sys.partition_range_values rv ON pf.function_id = rv.function_id
                                           AND p.partition_number = rv.boundary_id
LEFT JOIN sys.partition_range_values lv ON pf.function_id = lv.function_id
                                           AND p.partition_number - 1 = lv.boundary_id
JOIN sys.dm_db_partition_stats as stat on stat.object_id=p.object_id
    and stat.index_id=p.index_id
    and stat.index_id=p.index_id and stat.partition_id=p.partition_id
    and stat.partition_number=p.partition_number
JOIN sys.allocation_units as au on au.container_id = p.hobt_id
    and au.type_desc ='IN_ROW_DATA' 
        /* Avoiding double rows for columnstore indexes. */
        /* We can pick up LOB page count from partition_stats */
JOIN sys.filegroups as fg on fg.data_space_id = au.data_space_id
WHERE sc.name = @SchemaName
AND so.name = @TableName
AND p.partition_number >= @ShowPartNr
ORDER BY [Partition Function], [Partition #];
GO

(64 rows affected)

Total execution time: 00:00:00.678

Index,Rows,In-Row GB,Partition #,Compression,Partition Function,leftValue,rightValue,Boundary Type,Boundary Point (date),Filegroup
IX_Sales_Orders_OrderDate,89606,0.012,1,NONE,PF_OrderDate,NULL,20191001,Right / Lower,NULL,USERDATA
IX_Sales_Orders_OrderDate,45561,0.006,2,NONE,PF_OrderDate,20191001,20191101,Right / Lower,20191001,USERDATA
IX_Sales_Orders_OrderDate,43873,0.006,3,NONE,PF_OrderDate,20191101,20191201,Right / Lower,20191101,USERDATA
IX_Sales_Orders_OrderDate,45658,0.006,4,NONE,PF_OrderDate,20191201,20200101,Right / Lower,20191201,USERDATA
IX_Sales_Orders_OrderDate,45174,0.006,5,NONE,PF_OrderDate,20200101,20200201,Right / Lower,20200101,USERDATA
IX_Sales_Orders_OrderDate,42584,0.006,6,NONE,PF_OrderDate,20200201,20200301,Right / Lower,20200201,USERDATA
IX_Sales_Orders_OrderDate,45637,0.006,7,NONE,PF_OrderDate,20200301,20200401,Right / Lower,20200301,USERDATA
IX_Sales_Orders_OrderDate,43894,0.006,8,NONE,PF_OrderDate,20200401,20200501,Right / Lower,20200401,USERDATA
IX_Sales_Orders_OrderDate,45373,0.006,9,NONE,PF_OrderDate,20200501,20200601,Right / Lower,20200501,USERDATA
IX_Sales_Orders_OrderDate,44130,0.006,10,NONE,PF_OrderDate,20200601,20200701,Right / Lower,20200601,USERDATA
